In [ ]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import time
import json
import ast
from tqdm import tqdm
import requests
import warnings
warnings.filterwarnings('ignore') # turn off warnings

import psycopg2

connUAT = psycopg2.connect(
     host='localhost',
     database='uat',
     port=5431,
     user='postgres',
     password='1234567890'
  )

connPROC = psycopg2.connect(
     host='localhost',
     database='prod',
     port=5431,
     user='postgres',
     password='1234567890'
  )

cursorUAT = connUAT.cursor()
cursorPROC = connPROC.cursor()

def selectdf(query,conn):
    df = pd.read_sql(query, conn)
    return df
  
def SQL_push(sql_table_name,python_table,conn,cursor,inplace):
    python_table.replace([np.nan], [None],inplace=True)
    
    if inplace == True:
        try:
            sql_query = f'truncate table {sql_table_name}'
            cursor.execute(sql_query)
            conn.commit()
        except:
            pass
            
    sql = f"INSERT INTO {sql_table_name} VALUES ({','.join(['%s'] * len(python_table.columns))})"

    data = [tuple(row) for _, row in tqdm(python_table.iterrows(), total=len(python_table), desc="Preparing data")]

    # Chèn dữ liệu với xử lý lỗi
    for row in tqdm(data, total=len(data), desc="Inserting rows"):
        try:
            cursor.execute(sql, row)
        except Exception as e:
            print(f"Lỗi khi chèn dòng {row}: {e}")
            conn.rollback()  # Hủy giao dịch khi có lỗi, tránh PostgreSQL khóa transaction

    # Sau khi xong thì commit lại
    connUAT.commit()
    print("PUSH DATA: DONE")

pathfolder = r'F:\Work\Caohoc_2024_2026\PTPM_project\ml-latest'

In [22]:
sql_check = '''
            select distinct *
            from staging.stg_tmdb_json_movie_metadata stjmm 
            where request_json <> '{"success": false, "status_code": 34, "status_message": "The resource you requested could not be found."}'
            order by id
            '''
movie_json = selectdf(sql_check,connUAT)
# list_c['id'] = list_c['id'].astype(str)  # Đảm bảo cùng kiểu dữ liệu
movie_json.sort_values(by='id',inplace=True)
movie_json

,id,request_json
0,2,"{""adult"": false, ""backdrop_path"": ""/hQ4pYsIbP2..."
1,2,"{""adult"": false, ""backdrop_path"": ""/hQ4pYsIbP2..."
2,3,"{""adult"": false, ""backdrop_path"": ""/l94l89eMmF..."
3,3,"{""adult"": false, ""backdrop_path"": ""/l94l89eMmF..."
4,5,"{""adult"": false, ""backdrop_path"": ""/f2t4JbUvQI..."
...,...,...
85459,1144338,"{""adult"": false, ""backdrop_path"": null, ""belon..."
85460,1145358,"{""adult"": false, ""backdrop_path"": ""/4OIJmgez12..."
85461,1146246,"{""adult"": false, ""backdrop_path"": ""/t86SkVd35W..."
85462,1146249,"{""adult"": false, ""backdrop_path"": ""/eNopYAzvtf..."


In [23]:
data_dict_list = []

for i in tqdm(set(movie_json['id'])):
    data_str = movie_json[movie_json['id']==i]['request_json'].values[0]
    data = f'''{data_str}'''
    data_dict = json.loads(data)
    data_dict_list.append(data_dict)
    
movie_metadata = pd.DataFrame(data_dict_list)
movie_metadata.sort_values(by='id',inplace=True)
movie_metadata

100%|██████████| 85304/85304 [00:25<00:00, 3299.51it/s]


,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,origin_country,original_language,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
1,False,/hQ4pYsIbP22TMXOUdSfC2mjWrO0.jpg,"{'id': 1382526, 'name': 'Aki Kaurismäki's Prol...",0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",,2,tt0094675,[FI],fi,...,1988-10-21,0,73,"[{'english_name': 'Finnish', 'iso_639_1': 'fi'...",Released,,Ariel,False,7.111,346
2,False,/l94l89eMmFKh7na2a1u5q67VgNx.jpg,"{'id': 1382526, 'name': 'Aki Kaurismäki's Prol...",0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",,3,tt0092149,[FI],fi,...,1986-10-17,0,74,"[{'english_name': 'Finnish', 'iso_639_1': 'fi'...",Released,,Shadows in Paradise,False,7.295,410
3,False,/f2t4JbUvQIjUF5FstG1zZFAp02N.jpg,None,4000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 80, 'nam...",https://www.miramax.com/movie/four-rooms/,5,tt0113101,[US],en,...,1995-12-09,4257354,98,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,False,5.900,2683
4,False,/iUUpKunmBN5l8goObADBaFHnxQ8.jpg,None,21000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",,6,tt0107286,[US],en,...,1993-10-15,12136938,109,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Don't move. Don't whisper. Don't even breathe.,Judgment Night,False,6.458,349
5,False,/2w4xG178RpB4MDAIfTkqAuSJzec.jpg,"{'id': 10, 'name': 'Star Wars Collection', 'po...",11000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.starwars.com/films/star-wars-episod...,11,tt0076759,[US],en,...,1977-05-25,775398007,121,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,"A long time ago in a galaxy far, far away...",Star Wars,False,8.203,21056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49780,False,None,None,0,[],None,1144338,tt28212579,[],en,...,2023-08-11,0,0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,None,A House A Little Too Quiet,False,0.000,0
50074,False,/4OIJmgez12DITjHbWiNf1C7uG1d.jpg,None,0,"[{'id': 37, 'name': 'Western'}]",None,1145358,tt23469212,[US],en,...,2023-07-07,0,96,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Play your cards right and you just might survive.,Dead Man's Hand,False,5.778,9
50369,False,/t86SkVd35W9COtCGtbxxb3UoQpW.jpg,None,0,"[{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...",None,1146246,tt6810618,[CA],en,...,2024-08-22,0,84,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,None,The Deep Web: Murdershow,False,4.800,4
50371,False,/eNopYAzvtfvaj1yCI0AeVAOlHzU.jpg,None,0,"[{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...",None,1146249,tt17519798,[US],en,...,2023-07-12,0,88,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Insane in the membrane.,Cabin Girl,False,5.100,19


In [13]:
movie_metadata.columns.to_list()

['adult',
 'backdrop_path',
 'belongs_to_collection',
 'budget',
 'genres',
 'homepage',
 'id',
 'imdb_id',
 'origin_country',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'poster_path',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'title',
 'video',
 'vote_average',
 'vote_count',
 'success',
 'status_code',
 'status_message']

In [14]:
movie_metadata2 = movie_metadata[['id','title','original_title','original_language','release_date','status','overview','tagline','adult','popularity','homepage','poster_path','runtime','budget','revenue','vote_average','vote_count']]
movie_metadata2.head(5)

,id,title,original_title,original_language,release_date,status,overview,tagline,adult,popularity,homepage,poster_path,runtime,budget,revenue,vote_average,vote_count
1,2.0,Ariel,Ariel,fi,1988-10-21,Released,A Finnish man goes to the city to find a job a...,,False,1.5152,,/ojDg0PGvs6R9xYFodRct2kdI6wC.jpg,73.0,0.0,0.0,7.111,346.0
2,3.0,Shadows in Paradise,Varjoja paratiisissa,fi,1986-10-17,Released,"Nikander, a rubbish collector and would-be ent...",,False,1.2309,,/nj01hspawPof0mJmlgfjuLyJuRN.jpg,74.0,0.0,0.0,7.295,410.0
3,5.0,Four Rooms,Four Rooms,en,1995-12-09,Released,It's Ted the Bellhop's first night on the job....,Twelve outrageous guests. Four scandalous requ...,False,3.2229,https://www.miramax.com/movie/four-rooms/,/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg,98.0,4000000.0,4257354.0,5.859,2686.0
4,6.0,Judgment Night,Judgment Night,en,1993-10-15,Released,"Four young friends, while taking a shortcut en...",Don't move. Don't whisper. Don't even breathe.,False,1.7698,,/3rvvpS9YPM5HB2f4HYiNiJVtdam.jpg,109.0,21000000.0,12136938.0,6.458,349.0
5,11.0,Star Wars,Star Wars,en,1977-05-25,Released,Princess Leia is captured and held hostage by ...,"A long time ago in a galaxy far, far away...",False,19.8534,http://www.starwars.com/films/star-wars-episod...,/6FfCtAuVAW8XJjZ7eWeLibRLWTw.jpg,121.0,11000000.0,775398007.0,8.203,21056.0


In [15]:
movie_metadata2.columns.to_list()

['id',
 'title',
 'original_title',
 'original_language',
 'release_date',
 'status',
 'overview',
 'tagline',
 'adult',
 'popularity',
 'homepage',
 'poster_path',
 'runtime',
 'budget',
 'revenue',
 'vote_average',
 'vote_count']

In [16]:
movie_metadata[['id','belongs_to_collection']]

,id,belongs_to_collection
1,2.0,"{'id': 1382526, 'name': 'Aki Kaurismäki's Prol..."
2,3.0,"{'id': 1382526, 'name': 'Aki Kaurismäki's Prol..."
3,5.0,None
4,6.0,None
5,11.0,"{'id': 10, 'name': 'Star Wars Collection', 'po..."
...,...,...
86254,NaN,NaN
86277,NaN,NaN
86284,NaN,NaN
86290,NaN,NaN


### sử dụng:
- belongs_to_collection: dùng id để lọc những film liên quan lẫn nhau
- Dữ liệu có 126 movieid ko có mã tbdmid
- 1071 movieid có mã tbdmid những ko có dữ liệu trên server 
    - code: tbdm movies_info[movies_info['tmdbId'].isin(list(fall_m['id']))]


In [13]:
# tmdb_data = json.loads(sql_json[sql_json['id']==11]['request_json'].values[0])
# # tmdb_df = pd.DataFrame(tmdb_data)
# tmdb_data

In [ ]:
tmdb_data = json.loads(movie_metadata[movie_metadata['id']==1891]['request_json'].values[0])
# tmdb_df = pd.DataFrame(tmdb_data)
# tmdb_data

In [15]:
# [tmdb_data['production_countries'][0]['name'],tmdb_data['spoken_languages'][0]['english_name']
#  ,tmdb_data['production_companies'][0]['name']
#  ,tmdb_data['overview'],tmdb_data['status'],tmdb_data['release_date']
# ]

In [16]:
# movies_links[movies_links['tmdbId'].isnull()]

In [4]:
# movies = pd.read_csv(os.path.join(pathfolder,'movies.csv'))
links = pd.read_csv(os.path.join(pathfolder,'links.csv'))
links = links[links['tmdbId'].isnull()==False]
links.sort_values(by='tmdbId',inplace=True)
# movies_info = links.merge(movies, on='movieId', how='left')
# del movies_info['genres']
# del mo
# # movies.sort_values(by='tmdbId',inplace=True)
# movies_info
links


,movieId,imdbId,tmdbId
4366,4470,94675,2.0
12656,61724,92149,3.0
17,18,113101,5.0
474,479,107286,6.0
257,260,76759,11.0
...,...,...,...
86365,288447,28212579,1144338.0
86458,288733,23469212,1145358.0
86466,288753,6810618,1146246.0
86513,288913,17519798,1146249.0


In [38]:
movies_info[movies_info['movieId']==260]

,movieId,imdbId,tmdbId,title
4,260,76759,11.0,Star Wars: Episode IV - A New Hope (1977)


In [18]:
movies_info.isnull().sum()

movieId      0
imdbId       0
tmdbId     126
title        0
dtype: int64

In [19]:
# SQL_push(
#          sql_table_name='staging.stg_grouplen_movieinfo',
#          python_table=movies_info,
#          con=connUAT,
#          cursor=cursorUAT,
#          inplace= True
#         )

In [20]:
# DONE
movies = pd.read_csv(os.path.join(pathfolder,'movies.csv'))
movies1 = movies[['movieId','genres']]
max_col = [len(i) for i in list(movies1['genres'].str.split('|'))]
movies_genres = pd.DataFrame()

for i in range(max(max_col)):
    data = movies.copy()
    data['genres'] = data['genres'].str.split('|',expand=True)[i]
    data = data[data['genres'].isnull()==False]
    movies_genres = pd.concat([movies_genres,data])

movies_genres.sort_values(by='movieId',ascending=True,inplace=True)
del movies_genres['title']
movies_genres

,movieId,genres
0,1,Adventure
0,1,Fantasy
0,1,Animation
0,1,Comedy
0,1,Children
...,...,...
86533,288971,Horror
86534,288975,Documentary
86535,288977,Thriller
86535,288977,Crime


In [21]:
movies_genres.isnull().sum()

movieId    0
genres     0
dtype: int64

In [22]:
# SQL_push(
#          sql_table_name='staging.stg_grouplen_moviesgenres',
#          python_table=movies_genres,
#          con=connUAT,
#          cursor=cursorUAT,
#          inplace= True
#         )

In [ ]:
# DONE -- CSV
moives_tags = pd.read_csv(os.path.join(pathfolder,'tags.csv'))
# moives_tags['date_time'] = pd.to_datetime(moives_tags['timestamp'],unit='s')
# moives_tags['date'] = pd.to_datetime(moives_tags['date_time'],format='%Y-%m-%d %H:%M:%S').dt.strftime('%Y-%m-%d')
moives_tags['tag'] = moives_tags['tag'].str.lower().str.strip()
moives_tags['tag'] = moives_tags['tag'].replace('',np.nan)
del moives_tags['timestamp']
# tags.sort_values(by='tmdbId',inplace=True)
moives_tags

,userId,movieId,tag
0,10,260,good vs evil
1,10,260,harrison ford
2,10,260,sci-fi
3,14,1221,al pacino
4,14,1221,mafia
...,...,...,...
2328310,330923,176599,politically correct
2328311,330933,3317,coming of age
2328312,330933,3317,sexuality
2328313,330947,5782,not luc besson


In [ ]:
# moives_tags_agg = moives_tags[['movieId','tag']].drop_duplicates()
# moives_tags_agg = moives_tags_agg.groupby('movieId').agg(keywords=('tag',list)).reset_index()
# moives_tags_agg['keywords'] = moives_tags_agg['keywords'].astype('str')
# moives_tags_agg

,movieId,keywords
0,1,"['animation', 'friendship', 'toys', 'disney', ..."
1,2,"['animals', 'based on a book', 'fantasy', 'mag..."
2,3,"['sequel', 'moldy', 'old', 'old age', 'old men..."
3,4,"['characters', 'chick flick', 'girl movie', 'r..."
4,5,"['family', 'pregnancy', 'wedding', '4th wall',..."
...,...,...
53447,288765,"['post-apocalyptic', 'survival', 'tw suicide',..."
53448,288779,['don camillo series']
53449,288849,"['addiction', 'animation', 'short film']"
53450,288937,['anime']


In [54]:
# SQL_push(
#          sql_table_name='staging.stg_keywords_list',
#          python_table=moives_tags_agg,
#          con=connUAT,
#          cursor=cursorUAT,
#          inplace= True
#         )

In [40]:
test = moives_tags[moives_tags['movieId']==260][['movieId','tag']]
test.drop_duplicates(inplace=True)
test[test['tag'].str.contains('jedi')]

,movieId,tag
21775,260,jedi
136929,260,a boy grows into a jedi
525395,260,i wanna be a jedi
1060916,260,jedi master
1060917,260,jedi mind trick
1060918,260,jedi training
2310866,260,jedi wars


In [ ]:
moives_tags.isnull().sum()

userId        0
movieId       0
tag          27
date_time     0
dtype: int64

In [25]:
# SQL_push(
#             sql_table_name= 'staging.stg_grouplen_moviestags',
#             python_table= moives_tags,
#             con= connUAT,
#             cursor= cursorUAT,
#             inplace= True
#         )

In [26]:
# DONE
movies_ratings = pd.read_csv(os.path.join(pathfolder,'ratings.csv'))
movies_ratings['date_time'] = pd.to_datetime(movies_ratings['timestamp'],unit='s')
del movies_ratings['timestamp']
movies_ratings

,userId,movieId,rating,date_time
0,1,1,4.0,2008-11-03 17:52:19
1,1,110,4.0,2008-11-05 06:04:46
2,1,158,4.0,2008-11-03 17:31:43
3,1,260,4.5,2008-11-03 18:00:04
4,1,356,5.0,2008-11-03 17:58:39
...,...,...,...,...
33832157,330975,8340,2.0,2004-08-04 01:34:16
33832158,330975,8493,2.5,2004-08-04 02:15:09
33832159,330975,8622,4.0,2004-08-04 01:09:37
33832160,330975,8665,3.0,2004-08-04 01:09:25


In [27]:
movies_ratings.isnull().sum()

userId       0
movieId      0
rating       0
date_time    0
dtype: int64

In [28]:
# SQL_push(
#             sql_table_name= 'staging.stg_grouplen_moviesratings',
#             python_table= movies_ratings,
#             con= connUAT,
#             cursor= cursorUAT,
#             inplace= True
#         )

In [29]:
# DONE
genome_s = pd.read_csv(os.path.join(pathfolder,'genome-scores.csv'))
genome_s

,movieId,tagId,relevance
0,1,1,0.03200
1,1,2,0.02225
2,1,3,0.07000
3,1,4,0.05900
4,1,5,0.12300
...,...,...,...
18472123,288167,1124,0.09875
18472124,288167,1125,0.02950
18472125,288167,1126,0.02275
18472126,288167,1127,0.11225


In [30]:
genome_s.isnull().sum()

movieId      0
tagId        0
relevance    0
dtype: int64

In [31]:
# SQL_push(
#             sql_table_name= 'staging.stg_genome_scores',
#             python_table= genome_s,
#             con= connUAT,
#             cursor= cursorUAT,
#             inplace= True
#         )

In [32]:
# DONE
genome_t = pd.read_csv(os.path.join(pathfolder,'genome-tags.csv'))
genome_t

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s
...,...,...
1123,1124,writing
1124,1125,wuxia
1125,1126,wwii
1126,1127,zombie


In [33]:
genome_t.isnull().sum()

tagId    0
tag      0
dtype: int64

In [34]:
# SQL_push(
#             sql_table_name= 'staging.stg_genome_tags',
#             python_table= genome_t,
#             con= connUAT,
#             cursor= cursorUAT,
#             inplace= True
#         )

In [55]:
connUAT.commit()
connUAT.close()